# How Debuggers Work

Interactive _debuggers_ are tools that allow you to selectively observe the program state during an execution.  In this chapter, you will learn how such debuggers work – by building your own debugger.

**Prerequisites**

* You should have read the [Chapter on Tracing Executions](Tracer.ipynb).
* Again, knowing a bit of _Python_ is helpful for understanding the code examples in the book.

In [ ]:
import bookutils

In [ ]:
import sys

## Debuggers

_Interactive Debuggers_ (or short *debuggers*) are tools that allow you to observe program executions. A debugger typically offers the following features:

* _Run_ the program
* Define _conditions_ under which the execution should _stop_ and hand over control to the debugger. Conditions include
    * a particular location is reached
    * a particular variable takes a particular value
    * or some other condition of choice.
* When the program stops, you can _observe_ the current state, including
    * the current location
    * variables and their values
    * the current function and its callers
* When the program stops, you can _step_ through program execution, having it stop at the next instruction again.
* Finally, you can also _resume_ execution to the next stop.

This functionality often comes as a _command-line interface_, typing commands at a prompt; or as a _graphical user interface_, selecting commands from the screen. Debuggers can come as standalone tools, or be integrated into a programming environment of choice.

Debugger interaction typically follows a _loop_ pattern._ First, you identify the location(s) you want to inspect, and tell the debugger to stop execution once one of these _breakpoints_ is reached. Here's a command that could instruct a command-line debugger to stop at Line 239:

```
(debugger) break 239
(debugger) _
```

Then you have the debugger resume or start execution. The debugger will stop at the given location.

```
(debugger) continue
Line 239: s = x
(debugger) _
```

When it stops at the given location, you use debugger commands to inspect the state (and check whether things are as expected).

```
(debugger) print s
s = 'abc'
(debugger) _
```

You can then step through the program, executing more lines.

```
(debugger) step
Line 240: c = s[0]
(debugger) print c
c = 'a'
(debugger) _
```

You can also define new stop conditions, investigating other locations, variables, and conditions.

## Debugger Interaction

Let us now show how to build such a debugger.  The key idea of an _interactive_ debugger is to set up the _tracing function_ such that it actually _asks_ what to do next, prompting you to enter a _command_. For the sake of simplicity, we collect such a command interactively from a command line, using the Python `input()` function.

Our debugger holds a number of variables to indicate its current status:
* `stepping` is True whenever the user wants to step into the next line.
* `breakpoints` is a set of breakpoints (line numbers)
* `interact` is True while the user stays at one position.

We also store the current tracing information in three attributes `frame`, `event`, and `arg`.

In [ ]:
from Tracer import Tracer

In [ ]:
class Debugger(Tracer):
    """Interactive Debugger"""
    def __init__(self, file=sys.stdout):
        """Create a new interactive debugger."""
        self.stepping = True
        self.breakpoints = set()
        self.interact = True

        self.frame = None
        self.event = None
        self.arg = None

        super().__init__(file)

This is the main entry point for our debugger. If we should stop, we go into user interaction.

In [ ]:
class Debugger(Debugger):
    def traceit(self, frame, event, arg):
        """Tracing function; called at every line"""
        self.frame = frame
        self.event = event
        self.arg = arg

        if self.stop_here():
            self.interaction_loop()

        return self.traceit

We stop whenever we are stepping through the program or reach a breakpoint:

In [ ]:
class Debugger(Debugger):
    def stop_here(self):
        """Return true if we should stop"""
        return self.stepping or self.frame.f_lineno in self.breakpoints

Our interaction loop shows the current status, reads in commands, and executes them.

In [ ]:
class Debugger(Debugger):
    def interaction_loop(self):
        """Interact with the user"""
        self.print_debugger_status(self.frame, self.event, self.arg)

        self.interact = True
        while self.interact:
            command = input("(debugger) ")
            self.execute(command)

For a moment, let us implement two commands, `step` and `continue`. `step` steps through the program:

In [ ]:
class Debugger(Debugger):
    def step_command(self, arg=""):
        """Execute up to the next line"""
        self.stepping = True
        self.interact = False

In [ ]:
class Debugger(Debugger):
    def continue_command(self, arg=""):
        """Resume execution"""
        self.stepping = False
        self.interact = False

The `execute()` method dispatches between these two.

In [ ]:
class Debugger(Debugger):
    def execute(self, command):
        if command.startswith('s'):
            self.step_command()
        elif command.startswith('c'):
            self.continue_command()

Our debugger is now ready to run! We invoke it just like `Tracer`, using a `with` clause. The code

```python
with Debugger():
    remove_html_markup('abc')
```
gives us a debugger prompt
```
(debugger) _
```
where we can enter one of our two commands. Let us do two steps through the program and then resume execution:

In [ ]:
from Intro_Debugging import remove_html_markup

In [ ]:
from bookutils import input, next_inputs

In [ ]:
# ignore
next_inputs(["step", "step", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

Try this out for yourself by running the above invocation in the interactive notebook! If you are reading the Web version, the top menu entry `Resources` -> `Edit as Notebook` will do the trick. Navigate to the above invocation and press `Shift`+`Enter`.

### A Command Dispatcher

Our `execute()` function is still a bit rudimentary. A true command-line tool should provide means to tell which commands are available (`help`), automatically split arguments, and not stand in line of extensibility.

We therefore implement a better `execute()` method which does all that. Our revised `execute()` method _inspects_ its class for methods that end in `_command()`, and automatically registers their names as commands. Hence, with the above, we already get `step` and `continue` as possible commands.

### Excursion: Implementing execute()

Let us detail how we implement `execute()`. The `commands()` method returns a list of all commands (as strings) from the class.

In [ ]:
class Debugger(Debugger):
    def commands(self):
        cmds = [method.replace('_command', '') for method in dir(self.__class__) if method.endswith('_command')]
        cmds.sort()
        return cmds

In [ ]:
d = Debugger()
d.commands()

The `command_method()` method converts a given command (or its abbrevation) into a method to be called.

In [ ]:
class Debugger(Debugger):
    def command_method(self, command):
        if command.startswith('#'):
            return None  # Comment

        possible_cmds = [possible_cmd for possible_cmd in self.commands() if possible_cmd.startswith(command)]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None

        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')

In [ ]:
d = Debugger()
d.command_method("step")

In [ ]:
d = Debugger()
d.command_method("s")

The revised `execute()` method now determines this method and executes it with the given argument.

In [ ]:
class Debugger(Debugger):        
    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)

If `command_method()` cannot find the command, or finds more than one matching the prefix, it invokes the `help` command providing additional assistance. `help` draws extra info on each command from its documentation string.

In [ ]:
class Debugger(Debugger):        
    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""

        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands() if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                self.log(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                self.log(f"Ambiguous command {repr(command)}. Possible expansions are:")
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            self.log(f"{cmd:10} -- {method.__doc__}")


In [ ]:
d = Debugger()
d.execute("help")

In [ ]:
d = Debugger()
d.execute("foo")

### End of Excursion

## Printing Values

With `execute()`, we can now easily extend our class – all it takes is for a new command `NAME` is a new `NAME_command()` method. Let us start by providing a `print` command to print all variables. We use similar code as for the `Tracer` class in the [chapter on tracing](Tracer.ipynb).

In [ ]:
class Debugger(Debugger):
    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        vars = self.frame.f_locals
        self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))

In [ ]:
# ignore
next_inputs(["step", "step", "step", "print", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

Let us extend `print` such that if an argument is given, it only evaluates and prints out this argument.

In [ ]:
class Debugger(Debugger):
    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        vars = self.frame.f_locals

        if not arg:
            self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, globals(), vars))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")

In [ ]:
# ignore
next_inputs(["p s", "c"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

Note how we would abbreviate commands to speed things up. The argument to `print` can be any Python expression:

In [ ]:
# ignore
next_inputs(["print (s[0], 2 + 2)", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

Our `help` command also properly lists `print` as a possible command:

In [ ]:
# ignore
next_inputs(["help print", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

## Listing Source Code

We implement a `list` command that shows the source code of the current function.

In [ ]:
import inspect

In [ ]:
class Debugger(Debugger):
    def list_command(self, arg=""):
        """Show current function."""
        source_lines, line_number = inspect.getsourcelines(self.frame.f_code)
        current_line = self.frame.f_lineno

        for line in source_lines:
            self.log(f'{line_number:4} {line}', end='')
            line_number += 1

In [ ]:
# ignore
next_inputs(["list", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

## Setting Breakpoints

Stepping through the program line by line is a bit cumbersome. We therefore implement _breakpoints_ – a set of lines that cause the program to be interrupted as soon as this line is met.

In [ ]:
class Debugger(Debugger):
    def break_command(self, arg=""):
        """Set a breakoint in given line. If no line is given, list all breakpoints"""
        if arg:
            self.breakpoints.add(int(arg))
        self.log("Breakpoints:", self.breakpoints)

Here's an example, setting a breakpoint at the end of the loop:

In [ ]:
import inspect

In [ ]:
# ignore
_, remove_html_markup_starting_line_number = inspect.getsourcelines(remove_html_markup)
next_inputs([f"break {remove_html_markup_starting_line_number + 16}", 
             "continue", "print", "continue", "continue", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

In [ ]:
from bookutils import quiz

In [ ]:
quiz("What happens if we enter 'break 2 + 3'?",
     [
         "A breakpoint is set in Line 2.",
         "A breakpoint is set in Line 5.",
         "Two breakpoints are set in Lines 2 and 3.",
         "The debugger raises an exception."
     ], 1)

Try it out yourself by executing the above code block!

## Deleting Breakpoints

To delete breakpoints, we introduce a `delete` command:

In [ ]:
class Debugger(Debugger):
    def delete_command(self, arg=""):
        """Delete a breakoint in given line. If no line is given, clear all breakpoints"""
        if arg:
            try:
                self.breakpoints.remove(int(arg))
            except KeyError:
                self.log(f"No such breakpoint: {arg}")
        else:
            self.breakpoints = set()
        self.log("Breakpoints:", self.breakpoints)   

In [ ]:
# ignore
next_inputs([f"break {remove_html_markup_starting_line_number + 16}", 
             "continue", "print",
             f"delete {remove_html_markup_starting_line_number + 16}",
             "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

In [ ]:
quiz("What does the command 'delete' (without argument) do?",
    [
        "It deletes all breakpoints",
        "It deletes the source code",
        "It lists all breakpoints",
        "It stops execution"
    ],
     [n for n in range(2 // 2, 2 * 2) if n % 2 / 2]
    )

## Showing Breakpoints and Current Line

Let us extend `list` a bit such that 

1. it can also list a given function, and 
2. it shows the current line (`>`) as well as breakpoints (`#`)

In [ ]:
class Debugger(Debugger):
    def list_command(self, arg=""):
        """Show current function. If function is given, show its source code."""
        if arg:
            try:
                obj = eval(arg)
                source_lines, line_number = inspect.getsourcelines(obj)
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")
                return
            current_line = -1
        else:
            source_lines, line_number = inspect.getsourcelines(self.frame.f_code)
            current_line = self.frame.f_lineno

        for line in source_lines:
            spacer = ' '
            if line_number == current_line:
                spacer = '>'
            elif line_number in self.breakpoints:
                spacer = '#'
            self.log(f'{line_number:4}{spacer} {line}', end='')
            line_number += 1

In [ ]:
# ignore
_, remove_html_markup_starting_line_number = inspect.getsourcelines(remove_html_markup)
next_inputs([f"break {remove_html_markup_starting_line_number + 16}", 
             "list", "continue", "delete", "list", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

### Quitting

In the Python debugger interface, we can only observe, but not alter the control flow. To make sure we can always exit out of our debugging session, we introduce a `quit` command that deletes all breakpoints and resumes execution until the observed function finishes.

In [ ]:
class Debugger(Debugger):
    def quit_command(self, arg=""):
        """Finish execution"""
        self.breakpoints = []
        self.stepping = False
        self.interact = False

With this, our command palette is pretty complete, and we can use our debugger to happily inspect Python executions.

In [ ]:
# ignore
next_inputs(["help", "quit"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

## Synopsis

This chapter provides an interactive debugger for Python functions. The debugger is invoked as

```python
with Debugger():
    function_to_be_observed()
    ...
```
While running, you can enter _debugger commands_ at the `(debugger)` prompt. Here's an example session:

In [ ]:
# ignore
_, remove_html_markup_starting_line_number = inspect.getsourcelines(remove_html_markup)
next_inputs(["help", f"break {remove_html_markup_starting_line_number + 16}", 
             "list", "continue", "step", "print out", "quit"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

The `Debugger` class can be easily extended in subclasses. A new method `NAME_command(self, arg)` will be invoked whenever a command named `NAME` is entered, with `arg` holding given command arguments (empty string if none).

## Lessons Learned

* _Debugging hooks_ from interpreted languages allow for simple interactive debugging.
* A command-line debugging framework can be very easily extended with additional functionality.

## Next Steps

In the next chapter, we will see how to

* [locating defects by learning correlations between execution features and failures](DefectLocalizer.ipynb)


## Background

\todo{add}

## Exercises


### Exercise 1: Changing State

Some Python implementations allow to alter the state, by assigning values to `frame.f_locals`. Implement a `assign VAR=VALUE` command that allows to change the value of (local) variable `VAR` to the new value `VALUE`.

**Solution.** Assigning values may or may not work, depending on your Python implementation. Here's an `assign` command that gets things right, assuming we can assign to `frame.f_locals`.

In [ ]:
class Debugger(Debugger):
    def assign_command(self, arg):
        """Use as 'assign VAR=VALUE'. Assign local variable VAR the value VALUE."""
        sep = arg.find('=')
        if sep > 0:
            var = arg[:sep].strip()
            value = arg[sep + 1:].strip()
        else:
            self.help_command("assign")
            return
        
        try:
            self.frame.f_locals[var] = eval(value, globals(), self.frame.f_locals)
        except Exception as err:
            self.log(f"{err.__class__.__name__}: {err}")

This implementation allows us to _add_ variables, but not necessarily to _alter_ existing variables:

In [ ]:
# ignore
next_inputs(["assign b = 45", "assign s = 'xyz'", "print", "step", "print", "continue"])

In [ ]:
with Debugger():
    remove_html_markup('abc')

### Exercise 2: More Commands

Extending the `Debugger` class with extra features and commands is a breeze. The following commands are inspired from [the GNU command-line debugger (GDB)](https://www.gnu.org/software/gdb/):

#### Named breakpoints ("break")

With `break FUNCTION` and `delete FUNCTION`, set and delete a breakpoint at `FUNCTION`.

#### Step over functions ("next")

When stopped at a function call, the `next` command should execute the entire call, stopping when the function returns. (In contrast, `step` stops at the first line of the function called.)

#### Print call stack ("where")

Implement a `where` command that shows the stack of calling functions.

#### Move up and down the call stack ("up" and "down")

After entering the `up` command, explore the source and variables of the _calling_ function rather than the current function. Use `up` repeatedly to move further up the stack. `down` returns to the caller.

#### Execute until line ("until")

With `until LINE`, resume execution until a line greater than `LINE` is reached. If `LINE` is not given, resume execution until a line greater than the current is reached. This is useful to avoid stepping through multiple loop iterations.

#### Execute until return ("finish")

With `finish`, resume execution until the current fucnction returns.

#### Watchpoints ("watch")

With `watch CONDITION`, stop execution as soon as `CONDITION` changes its value. (Use the code from our `EventTracer` class in the [chapter on Tracing](Tracer.ipynb).) `delete CONDITION` removes the watchpoint. Keep in mind that some variable names may not exist at all times.

### Exercise 3: Time-Travel Debugging

Rather than inspecting a function at the moment it executes, you can also _record_ the entire state (call stack, local variables, etc.) during execution, and then run an interactive session to step through the recorded execution. This allows for a `back` command which gets you back to earlier states.